# Restaurant Recommendation

In [15]:
import requests
from bs4 import BeautifulSoup

In [16]:
response = requests.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc=Maryville%2C+MO+64468")

In [19]:
soup = BeautifulSoup(response.content, "html.parser")

In [36]:
card_divs = soup.find_all('div', class_='css-33bb9k', attrs={'data-testid': 'serp-ia-card'})
num_cards = len(card_divs)
print(num_cards)

10


In [139]:

business_ratings = {}
business_review_urls = {}

base_url = "https://www.yelp.com"

for card_div in card_divs:
    # Extract business name
    name = card_div.find('a', class_='css-19v1rkv').text.strip()
    print("Business Name:", name)
    more_span = card_div.find('span', class_='css-1wayfxy')
    if more_span:
        more_link = more_span.find('a')
        if more_link:
            review_url = more_link['href']
            full_review_url = base_url +  review_url
            print("Link to full review:", full_review_url)
            business_review_urls[name] = full_review_url
        else:
            print("Full review link not found")
    else:
        print("Span tag containing the link not found")

    # Find the total ratings
    total_ratings_element = card_div.find('span', class_='css-gutk1c')
    if total_ratings_element:
        total_ratings = total_ratings_element.text.strip()
        print("Total Ratings:", total_ratings)
    else:
        total_ratings = 0
        print("Total Ratings", total_ratings)
        print("Total Ratings not found")

    business_ratings[name] = total_ratings

    print()  

Business Name: A&G Restaurant
Link to full review: https://www.yelp.com/biz/a-and-g-restaurant-maryville?hrid=6HiKKch4iVUvNLQrsl_RiA&osq=Restaurants
Total Ratings: 3.9

Business Name: William Coy’s Farm to Table Restaurant
Link to full review: https://www.yelp.com/biz/william-coys-farm-to-table-restaurant-maryville?hrid=gyKkuYOmCWgnWzfPwkN46Q&osq=Restaurants
Total Ratings: 3.7

Business Name: Pagliai’s Pizza
Link to full review: https://www.yelp.com/biz/pagliais-pizza-maryville?hrid=4uXy43ISkUhRy5sSwoSCXQ&osq=Restaurants
Total Ratings: 4.3

Business Name: Happy Garden
Link to full review: https://www.yelp.com/biz/happy-garden-maryville?hrid=TNLbHpOeZcXbDnN2te1dbw&osq=Restaurants
Total Ratings: 4.1

Business Name: Simply Siam
Link to full review: https://www.yelp.com/biz/simply-siam-maryville-2?hrid=rVrbr1EiprHQX2ecKw2oMw&osq=Restaurants
Total Ratings: 4.1

Business Name: El Maguey
Link to full review: https://www.yelp.com/biz/el-maguey-maryville?hrid=4UU6TDsOrjKv9X7ny8Lwdw&osq=Restaura

# Manipulating data to get the name of the restaurant, total ratings and link to reviews of respective restaurant
 - Name of restaurant and it's rating to visualize data for recommendation
 - Link to full review to extract and anlalyzize the reviews for each restaurant

In [138]:
for key, value in business_review_urls.items():
    print(key, ":", value)

A&G Restaurant : https://www.yelp.com/biz/chick-fil-a-maryville-3?hrid=fCLihQ2IaYYQC1BuU6RMJw&osq=Restaurants
William Coy’s Farm to Table Restaurant : https://www.yelp.com/biz/chick-fil-a-maryville-3?hrid=fCLihQ2IaYYQC1BuU6RMJw&osq=Restaurants
Pagliai’s Pizza : https://www.yelp.com/biz/chick-fil-a-maryville-3?hrid=fCLihQ2IaYYQC1BuU6RMJw&osq=Restaurants
Happy Garden : https://www.yelp.com/biz/chick-fil-a-maryville-3?hrid=fCLihQ2IaYYQC1BuU6RMJw&osq=Restaurants
Simply Siam : https://www.yelp.com/biz/chick-fil-a-maryville-3?hrid=fCLihQ2IaYYQC1BuU6RMJw&osq=Restaurants
El Maguey : https://www.yelp.com/biz/chick-fil-a-maryville-3?hrid=fCLihQ2IaYYQC1BuU6RMJw&osq=Restaurants
Planet Sub : https://www.yelp.com/biz/chick-fil-a-maryville-3?hrid=fCLihQ2IaYYQC1BuU6RMJw&osq=Restaurants
Pizza Ranch : https://www.yelp.com/biz/chick-fil-a-maryville-3?hrid=fCLihQ2IaYYQC1BuU6RMJw&osq=Restaurants
Chick-fil-A : https://www.yelp.com/biz/chick-fil-a-maryville-3?hrid=fCLihQ2IaYYQC1BuU6RMJw&osq=Restaurants
